# Spherical Code

With both X and Y trajectories right now!

# Load the data

In [2]:
%load_ext autoreload
%autoreload 2

import pickle
import numpy as np 
from matplotlib import pyplot as plt

%matplotlib inline

Load the trajectories

In [3]:
with open('prepare_trajectories.pkl', 'rb') as handle:
    pkl_obj = pickle.load(handle)
    container_t_np = pkl_obj['t']
    container_x_np = pkl_obj['x']
    container_y_np = pkl_obj['y']
    print('Data loaded from file')

num_of_trajs = container_y_np.shape[0]

Data loaded from file


Load the prior parameters

In [6]:
with open('y.pkl', 'rb') as handle:
    pkl_obj = pickle.load(handle)
    mean_y = pkl_obj['mean_curve']
    gm_y_mean = pkl_obj['mu']
    gm_y_cov = pkl_obj['cov']
    y_basis = pkl_obj['basis']
print('Prior for longitudinal trajectories loaded')

with open('x.pkl', 'rb') as handle:
    pkl_obj = pickle.load(handle)
    mean_x = pkl_obj['mean_curve']
    gm_x_mean = pkl_obj['mu']
    gm_x_cov = pkl_obj['cov']
    x_basis = pkl_obj['basis']
print('Prior for lateral trajectories loaded')


Prior for longitudinal trajectories loaded
Prior for lateral trajectories loaded


# Some helper functions

In [7]:
from scipy.stats import norm 
def likelihood(z, theta, t, x_or_y = 'y'):
    friendly_basis = y_basis
    if x_or_y == 'x':
        friendly_basis = x_basis
    reconstructed_trajectory = theta.T @ friendly_basis 
    #print('Mean: ', reconstructed_trajectory[t])
    #print('Test: ', z)
    #raise Exception("LALALA")
    return norm.pdf(z, loc=reconstructed_trajectory[t], scale=3.0) # scale is the std

In [8]:
# Low variance resampling
def lvr(particles, weights): # requires normalized weights first
    # return: new particles with equal weights
    M = particles.shape[0] # number of particles
    new_particles = np.zeros_like(particles)
    # new_particles_weight = np.ones((M,)) / M 
    # W = np.sum(weights)
    r = np.random.rand() / M # get a random number between 0 and 1/M
    c = weights[0]
    i = 0
    for m in range(M):
        U = r + m/M
        while U>c:
            i+=1
            c+=weights[i]
        new_particles[m] = particles[i]
    return new_particles

# Relative Inferential Spherical Coding for Y component (RISC)

First of all, the whole idea is similar to delta compression: as the estimation of the representation vector $a$ gets better with observations arriving, we will subsequently broadcast the change to other vehicles, which is the delta compared to the last estimate. All MAP estimates start from the prior's mean $\overline{a}$, then gradually moves to the corresponding generating representation vector $a^*$.

Encoding the radial component should be simple: we send a 1 if we want to move to the next annulus of lattice in the outward radial direction. We send a 0 if we want to move in inward radial direction. Overall, the less common the trajectory is, the more bits we have to send (in multiple attempts) to other vehicles. Vice-versa, because most trajectories can be represented close to the mean, we expect to send little bits to determine the "correct" annulus $a^*$ is in.

In [10]:
import sys 

sys.path.append('C:\\Users\\nxf67027\\Documents\\anomaly-detection-ngsim')

In [11]:
from scipy.stats import norm 
from coding import huffman
import sys 

In [12]:
def angular_minus(a,b):
    d = np.mod(a - b, np.pi * 2)
    return (d > np.pi) * (np.pi * 2 - d) + (d <= np.pi) * d

We are going to test against multiple trajectories, each trajectory will have 2 configurations: a uniform distribution and a spherical code distribution

This is the code for one configuration, we have to copy it and run two times.

In [ ]:
from scipy.stats import multivariate_normal

multivariate_normal.rvs(mean = gm_y_mean, cov = gm_y_cov)

In [13]:
np.random.seed(6969)
test_trajectory_index = np.random.randint(container_y_np.shape[0], size=10)
# To store the test results
test_bits_log = np.zeros((test_trajectory_index.shape[0], 140)) # set 140 to y_test below too
recons_err_log = np.zeros((test_trajectory_index.shape[0], 140))
test_configuration = 'spherical' # spherical or uniform

for test_no, y_test_id in enumerate(test_trajectory_index.tolist()):
    # generate the test trajectory
    y_test = container_y_np[y_test_id,:140]
    print('Test number {}/{} (trajectory {}) '.format(test_no + 1, test_trajectory_index.shape[0], y_test_id))

    ###

    n_particles = 5000 
    map_particle_available = False

    # Step 1: generate a bunch of particles from the prior distribution
    particles = kde.sample(n_particles, random_state=6969)
    weights = np.ones((n_particles,)) / n_particles
    map_components_log = np.zeros((y_test.shape[0], particles.shape[1]))
    weights_variance_log = np.zeros((y_test.shape[0],))
    measurement_likelihood_log = np.zeros((y_test.shape[0],))
    error_log = np.zeros((y_test.shape[0],)) # the error we observe (due to MAPE has changed)
    reconstruction_error_log = np.zeros_like(error_log) # the error our clients will observe
    broadcast_log = np.zeros((y_test.shape[0],))
    bits_log = []
    last_transmission_at_time = 0
    # Some debuggin variables
    beta_sd_history = np.zeros((y_test.shape[0], 4))
    betac_history = np.zeros((y_test.shape[0], 4))


    lattice_size = 1/np.sqrt(np.shape(particles[0])[0]) # 1/sqrt(5) in case of 5 basis components

    for t in range(y_test.shape[0]):
        # When measurement arrives, calculate the importance weights first
        weights_normalization_coeff = 0
        for j in range(weights.shape[0]):
            measurement_likelihood = likelihood(y_test[t], particles[j,:], t)
            weights[j] = measurement_likelihood
            weights_normalization_coeff += weights[j]
            measurement_likelihood_log[t] += measurement_likelihood

        # print(t, measurement_likelihood)

        # Normalization all the weights
        for j in range(weights.shape[0]):
            weights[j] = weights[j] / weights_normalization_coeff
        
        if not map_particle_available: # first assignment of map_particle_value
            # Get the index of the particle with largest weight, that will be our maximum-a-posteriori estimate 
            map_particle_index = np.argmax(weights)
            map_particle_value = particles[map_particle_index]
            mean_particle = map_particle_value.copy()
            map_components_log[t,:] = map_particle_value
            map_particle_available = True

            # Intialization of variables at the beginning of the inference process
            alpha = np.zeros_like(mean_particle)
            # alphac = np.zeros_like(alpha)
            beta = np.zeros((np.shape(mean_particle)[0]-1,))
            beta_mean = beta.copy() # this is the smoothed out mean of the beta vector using beta in the past for which we will use to build the Huffman's probability table 
            # beta_sd = np.ones_like(beta_mean) * np.pi / 4 # this is the variance vector to accompany with the mean vector above
            beta_sd = np.ones_like(beta_mean) * np.pi * 4 # starting from a more uniform distribution
            betaq = np.zeros_like(beta)
            rq = 0 
            phi = np.zeros_like(beta)
            phiq = np.zeros_like(beta)
            map_particle_value_c = map_particle_value.copy()
            
            
        error_log[t] = np.abs(((map_particle_value_c) @ friendly_basis)[t] - y_test[t])
        
        map_particle_index = np.argmax(weights)
        map_particle_value_new = particles[map_particle_index]
        
        if error_log[t] > 3.0: # if the prediction from MAP deviates up to 2ft from the actual observations
            # we will have to broadcast an update so that our fellows can adjust accordingly
            alpha_new = map_particle_value_new - mean_particle
            delta = alpha_new - alpha 
            
            # calculating radial component
            delta_r = np.linalg.norm(delta) # change in radius
            delta_rq = np.floor(delta_r / lattice_size) # quantized change in radius
            delta_rc = delta_rq * lattice_size # reconstructed change in radius 
            r_bits = delta_rq
            if delta_rc == 0: 
                delta_rc = 1
                r_entropy = 1
            else:
                r_entropy = np.log2(delta_rq) 

            last_transmission_at_time = t


            # calculating the spherical coordinates (i.e., the beta vector)
            beta_new = np.zeros((np.shape(mean_particle)[0]-1,))
            for dimension in range(np.shape(mean_particle)[0]-1):
                # formula for spherical coordinates (first 3 components)
                beta_new[dimension] = np.arccos(delta[dimension]/np.linalg.norm(delta[dimension:]))
            # spherical coordinate for the final component
            if delta[-1] >= 0:
                beta_new[-1] = np.arccos(delta[-2]/np.linalg.norm(delta[-2:]))
            else:
                beta_new[-1] = 2 * np.pi - np.arccos(delta[-2]/np.linalg.norm(delta[-2:]))

            # quantizing the spherical coordinates
            number_of_lattices = np.floor(2 * np.pi * delta_rc / lattice_size) + 1
            phi_new = np.zeros_like(beta_new)
            betac_new = np.zeros_like(phi_new)
            for dimension in range(np.shape(mean_particle)[0]-1):
                phi_new[dimension] = np.floor(delta_rc * beta_new[dimension] / lattice_size) # to be encoded with Huffman
                betac_new[dimension] = phi_new[dimension] * lattice_size / delta_rc # to be used with rc to reconstruct the alphac vector

            # reconstruct the alpha vector (the thing our clients will get)
            deltac = np.zeros_like(alpha_new)
            for i in range(np.shape(mean_particle)[0]-1):
                v = delta_rc
                for k in range(i):
                    v = v * np.sin(betac_new[k])
                v = v * np.cos(betac_new[i])  
                deltac[i] = v 
            deltac[-1] = deltac[-2] / np.cos(betac_new[-1]) * np.sin(betac_new[-1])

            # Reconstruction of the MAP particle from what the client receives
            alphac_new = alpha + deltac # this is what our client will get
            map_particle_value_c = alphac_new + mean_particle # this is the particle our client will use to extrapolate the trajectory 

            # print('Alpha distance: ', np.linalg.norm(alphac_new - alpha_new))

            # encode the phi_new vector 
            phi_entropy = 0
            phi_bits = np.zeros_like(phi_new)
            phi_prob = np.zeros_like(phi_bits)
            for dimension in range(np.shape(mean_particle)[0]-1):
                # first, we need to create a probability table to derive Huffman's code
                if number_of_lattices < 2: # ensure there will be a minimum of 2 lattices to encode
                    p_lattice = np.ones((2,))
                else:
                    p_lattice = np.zeros((int(number_of_lattices),))
                for i in range(int(number_of_lattices)):
                    if test_configuration == 'spherical':
                        p_lattice[i] = norm.pdf(angular_minus(2 * np.pi / number_of_lattices * i, beta_mean[dimension]), 0, beta_sd[dimension])
                    elif test_configuration == 'uniform':
                        p_lattice[i] = 1 # uniform
                    else:
                        raise Exception('test_configuration is not set')
                if np.linalg.norm(p_lattice) < 1e-2:
                    p_lattice = np.ones_like(p_lattice) # preventing p_lattice from being too small
                p_lattice = p_lattice / np.linalg.norm(p_lattice)
                # generate the Huffman code (see Useful Snippet 1 below)
                p_lattice_sort_arg = np.argsort(p_lattice)[::-1]
                p_lattice_sorted = p_lattice[p_lattice_sort_arg]
                p_lattice_sort_map = np.argsort(p_lattice_sort_arg)
                # generate 
                code = huffman.HuffmanCode(p_lattice_sorted.tolist())
                spherical_code = code.compute_code()

                for index, s in enumerate(spherical_code):
                    if s == '':
                        spherical_code[index] = '0'

                phi_prob[dimension] = p_lattice_sorted[p_lattice_sort_map[int(phi_new[dimension])]]
                phi_bits[dimension] = int(spherical_code[p_lattice_sort_map[int(phi_new[dimension])]], 2) # convert a string of binary to int
                if phi_bits[dimension]<1e-2:
                    phi_bits[dimension] += 0
                else:
                    phi_entropy += np.log2(phi_bits[dimension])

            # Update beta_mean and beta_sd
            angular_dist = angular_minus(betac_new, beta_mean)
            beta_mean = 0.3 * beta_mean + 0.7 * betac_new 
            beta_sd = 0.7 * (np.abs(angular_dist) * 1.96) + 0.3 * beta_sd

            # print('Radial/angular delta vector: {} / {}'.format(r_bits, phi_bits))
            if False: #silencing the logging process
                print('Codeword proba: ', phi_prob)
                print('Ang dist btwn trans (deg): {}'.format(angular_dist / np.pi * 180))
                print('Rad bits / Ang bits: {:.2f}/{:.2f}'.format(r_entropy, phi_entropy))
                #print('W/O bef/aft cmpr: {:.0f} -> {:.0f} ({:.2f}%) bits'.format(5 * 32, r_entropy + phi_entropy, (r_entropy + phi_entropy) / (5 * 64) * 100))
                print('--- END OF TRANSMISSION ATTEMPT ---')
            

            # Assign new values to current values
            # map_particle_value = map_particle_value_new_c.copy() # should reconstruct alpha from quantized parameters and place it here
            alpha = alphac_new.copy()
            beta = betac_new.copy() # reconstructed from quantization 
            phi = phi_new.copy()  # reconstructed from quantization too
            map_components_log[t,:] = map_particle_value
            broadcast_log[t] = 1
            bits_log.append(r_entropy + phi_entropy)
            # logging for debugging
            beta_sd_history[t,:] = beta_sd
            betac_history[t,:] = betac_new

            test_bits_log[test_no, t] = r_entropy + phi_entropy # for batch testing log

            # sys.exit()

        reconstruction_error_log[t] = np.abs(((map_particle_value_c) @ friendly_basis)[t] - y_test[t])
        recons_err_log[test_no, t] = reconstruction_error_log[t] 
        
        if reconstruction_error_log[t] > 2.00:
            # raise Exception("While this error is so high?")
            pass 
        
        # Resample the particles according to the particle weights
        if measurement_likelihood_log[t] < 100:
            particles = lvr(particles, weights)
            weights = np.ones((n_particles,)) / n_particles

        weights_variance_log[t] = np.var(weights)


    ###
    
    

NameError: name 'y_vec' is not defined